In [ ]:
%reset -f

import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sns
import csv
import re
import sys, os, random
import gc
sys.path.insert(0, "e:/xgboost/python-package")
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

#root = '/Users/schwalmdaniel/github/kaggle/house_prices_kaggle'
root = 'e:/kaggle/avito-demand-prediction'

train=pd.read_csv(root + "/train.csv")
test=pd.read_csv(root + "/test.csv")

# have a look at the ds
train.head()

In [ ]:
# dummify columns
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset, columns = ['user_type','parent_category_name','category_name'], 
               prefix_sep='__')
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
del dataset
train.head()

In [ ]:
lbl = LabelEncoder()
train["city"] = lbl.fit_transform(train["city"].astype(str))
test["city"] = lbl.fit_transform(test["city"].astype(str))
train["region"] = lbl.fit_transform(train["region"].astype(str))
test["region"] = lbl.fit_transform(test["region"].astype(str))
train.head()

In [ ]:
columnsToDrop = ['item_id','user_id','description','image','image_top_1','price','param_1',
                'param_2','param_3','title','activation_date']
train = train.drop(columnsToDrop,axis=1,errors='ignore') 
test2 = test.drop(columnsToDrop,axis=1,errors='ignore') 


In [ ]:
X = train.drop(['deal_probability'],axis=1,errors='ignore') 
y = train['deal_probability']
X_test = test2.drop(['deal_probability'],axis=1,errors='ignore') 

del train

gc.collect()

In [ ]:
def _get_xgb_feat_importances(clf, feature_names):

        try:
            # Handles case when clf has been created by calling
            # xgb.XGBClassifier.fit() or xgb.XGBRegressor().fit()
            fscore = clf.get_booster().get_fscore()
        except:
            # Handles case when clf has been created by calling xgb.train.
            # Thus, clf is an instance of xgb.Booster.
            fscore = clf.get_fscore()

        trained_feature_names = feature_names #self._get_trained_feature_names()

        feat_importances = []
        
        #print(fscore.items())

        features = {}
        
        for k, v in fscore.items():
            features[k] = v
            
        return sorted(features.items(), key=lambda x: x[1])[-50:]
    
def rmse_cv(model, train,y):
    n_folds = 5
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y, scoring="neg_mean_squared_log_error", cv = kf))
    return(rmse)

In [ ]:
model = XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=200,#2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

#pipeline = Pipeline([('imputer', Imputer(strategy='mean')), ('classify', model)])
pipeline = Pipeline([('classify', model)])
pipeline.fit(X,y)
score = rmse_cv(pipeline, X, y)
print("\pipeline score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

preds = pipeline.predict(X_test)

predicted = pd.DataFrame()
predicted['item_id'] = test['item_id']
predicted['deal_probability'] = preds
predicted.to_csv(root + '/submission.csv', index=False,quoting=csv.QUOTE_NONNUMERIC)

predicted.head()

In [ ]:
str(list(_get_xgb_feat_importances(model, X.columns.tolist())))

In [ ]:
subm = pd.DataFrame()
subm['item_id'] = test['item_id']
subm['deal_probability'] = preds
subm.to_csv(root + '/submission.csv', index=False,quoting=csv.QUOTE_NONNUMERIC)
subm.head()